<a href="https://colab.research.google.com/github/ygebre1/us-migration-flow-analysis-and-visualization/blob/trial1/us_migration_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U.S. Migration Trends from 2010 to 2023

This project leverages data from the U.S. Census Bureau, covering State-to-State migration trends over a 13-year span, from 2010 to 2023. The dataset provides key insights into the population distribution across states, the number of people who resided in a different state the previous year, and the number of individuals who were living abroad during the same period.

## Initial Overview of the Dataset
The dataset consists of 13 Excel files containing migration data from 2010 to 2023, excluding 2020, as the data for that year was not provided by the U.S. Census Bureau at [this link](https://www.census.gov/data/tables/time-series/demo/geographic-mobility/state-to-state-migration.html). The format of each file includes multiple headers and comments, but the number of rows and columns is consistent across all files.

## Data Exploration
Below are the steps for exploring the dataset:

- Clean the dataset by removing titles and comments.
- Remove the MOE columns from all files.
- Set row 6 as the header and rename the columns accordingly.
- Combine all the data files and reshape them into a long format.

Let's begin by importing the required libraries, loading the data, and inspecting the dataset.

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# Import necessary libraries
import pandas as pd
import numpy as np
import glob
import re
import os

# Define the path to the data folder in Google Drive
data_folder_path = '/content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data'

# Use glob to find all Excel files (.xls and .xlsx) in the folder
migration_files = glob.glob(data_folder_path + '/*.xls*')

# Initialize an empty list to store processed dataframes
processed_dataframes = []

# Loop through each Excel file in the folder
for migration_file in migration_files:
    print(f"Processing file: {migration_file}")

    # Read the Excel file into a dataframe, skip titles and comments, and fill missing values with 0
    migration_data = pd.read_excel(migration_file, na_values=['(NA)'], header=6).fillna(0)

    # Remove rows related to footnotes (rows 70 to 76) and columns containing "MOE" values
    migration_data = migration_data.loc[:70, (migration_data != 'MOE').all()]

    # Rename columns for clarity
    migration_data.rename(columns={
        migration_data.columns[0]: 'destination_state',        # State where people moved to
        migration_data.columns[1]: 'population',                # Population in the destination state
        migration_data.columns[2]: 'same_house',                # People who stayed in the same house
        migration_data.columns[3]: 'same_state',                # People who stayed in the same state
        migration_data.columns[4]: 'from_different_state_Total', # People who moved from a different state
        migration_data.columns[-4]: 'abroad_Total',             # People who moved abroad (total)
        migration_data.columns[-3]: 'abroad_PuertoRico',        # People who moved to Puerto Rico
        migration_data.columns[-2]: 'abroad_USIslandArea',      # People who moved to U.S. Island Areas
        migration_data.columns[-1]: 'abroad_ForeignCountry'     # People who moved to foreign countries
    }, inplace=True)

    # Remove rows with missing destination state and drop unnecessary columns
    migration_data = migration_data[migration_data['destination_state'] != 0]
    migration_data = migration_data[migration_data.columns.drop(list(migration_data.filter(regex='Unnamed:')))]
    migration_data = migration_data.drop([2])  # Drop irrelevant row with index 2

    # Convert all numerical columns to integers
    for column in migration_data.columns[1:]:  # Skip the first column (state names)
        migration_data[column] = pd.to_numeric(migration_data[column], errors='coerce').fillna(0).astype(int)

    # Extract the year from the file name and create a new 'year' column
    migration_data['year'] = re.findall('\d+', os.path.basename(migration_file))[0]

    # Reset the index for the dataframe
    migration_data.reset_index(drop=True, inplace=True)

    # Append the processed dataframe to the list
    processed_dataframes.append(migration_data)

# Display the first 5 rows of the first processed file
processed_dataframes[0].head()


Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/state_to_state_migrations_table_2010.xls
Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/state_to_state_migrations_table_2011.xls
Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/state_to_state_migrations_table_2013.xls
Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/state_to_state_migrations_table_2012.xls
Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/State_to_State_Migrations_Table_2015.xls
Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/State_to_State_Migrations_Table_2014.xls
Processing file: /content/drive/My Drive/Colab Notebooks/migrations_2010_to_2023/raw_migrations_data/State_to_State_Migrations_Table_2016.xls
Proces

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,4729509,3987155,620465,108723,0,3013,676,1481,3827,...,2490,1171,41,1155,27,13166,228,181,12757,2010
1,Alaska,702974,565031,95878,36326,477,0,1354,47,3906,...,714,2421,0,158,81,5739,19,1374,4346,2010
2,Arizona,6332786,5069002,1001991,222725,416,3109,0,689,47164,...,3413,12645,595,5556,593,39068,599,223,38246,2010
3,Arkansas,2888304,2387806,412997,79127,1405,934,777,0,4457,...,494,264,0,821,443,8374,87,0,8287,2010
4,California,36907897,30790221,5413287,444749,3364,9579,33854,4172,0,...,14232,30544,1446,6031,1336,259640,1223,5123,253294,2010


Combine all dataframes and reshape them into long format.

In [66]:
# Concatenate all processed dataframes into a single dataframe
combined_migration_data = pd.concat(processed_dataframes)

# Display the combined dataframe
combined_migration_data

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year,Total,Total.1,Puerto Rico
0,Alabama,4729509,3987155,620465,108723,0,3013,676,1481,3827,...,1155,27,13166,228,181,12757,2010,NaN,NaN,NaN
1,Alaska,702974,565031,95878,36326,477,0,1354,47,3906,...,158,81,5739,19,1374,4346,2010,NaN,NaN,NaN
2,Arizona,6332786,5069002,1001991,222725,416,3109,0,689,47164,...,5556,593,39068,599,223,38246,2010,NaN,NaN,NaN
3,Arkansas,2888304,2387806,412997,79127,1405,934,777,0,4457,...,821,443,8374,87,0,8287,2010,NaN,NaN,NaN
4,California,36907897,30790221,5413287,444749,3364,9579,33854,4172,0,...,6031,1336,259640,1223,5123,253294,2010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,Washington,7710339,6572952,831247,248355,488,3048,7237,1105,49968,...,823,253,57785,195,2884,54706,2022,NaN,NaN,NaN
49,West Virginia,1758432,1576119,136161,43493,478,0,240,121,879,...,192,170,2659,0,0,2659,2022,NaN,NaN,NaN
50,Wisconsin,5835492,5119356,571451,120434,635,774,3474,694,7537,...,0,345,24251,1864,108,22279,2022,NaN,NaN,NaN
51,Wyoming,575705,487359,57696,28948,0,0,2132,57,1558,...,369,0,1702,4,221,1477,2022,NaN,NaN,NaN


In [67]:
# Define the columns that will remain as identifiers (e.g., state, year, and migration categories)
identifier_columns = ['destination_state', 'year', 'population', 'same_house', 'same_state', 'from_different_state_Total', 'abroad_Total']

# Identify the remaining columns (those that represent migration flows from various states)
migration_columns = [col for col in combined_migration_data.columns if col not in identifier_columns]

# Reshape the data into a long format using pd.melt, with migration categories as columns
long_format_migration_data = pd.melt(combined_migration_data, id_vars=identifier_columns, value_vars=migration_columns, var_name='migration_type', value_name='number_of_people')


In [68]:
'''
Convert the 'year' column from object type to integer (if necessary)
This step ensures the 'year' column is in the correct format for analysis
'''
# long_format_migration_data['year'] = pd.to_datetime(long_format_migration_data['year'], format='%Y').dt.year


"\nConvert the 'year' column from object type to integer (if necessary)\nThis step ensures the 'year' column is in the correct format for analysis\n"

In [69]:
long_format_migration_data

,destination_state,year,population,same_house,same_state,from_different_state_Total,abroad_Total,migration_type,number_of_people
0,Alabama,2010,4729509,3987155,620465,108723,13166,Alabama,0.0
1,Alaska,2010,702974,565031,95878,36326,5739,Alabama,477.0
2,Arizona,2010,6332786,5069002,1001991,222725,39068,Alabama,416.0
3,Arkansas,2010,2888304,2387806,412997,79127,8374,Alabama,1405.0
4,California,2010,36907897,30790221,5413287,444749,259640,Alabama,3364.0
...,...,...,...,...,...,...,...,...,...
39781,Washington,2022,7710339,6572952,831247,248355,57785,Puerto Rico,NaN
39782,West Virginia,2022,1758432,1576119,136161,43493,2659,Puerto Rico,NaN
39783,Wisconsin,2022,5835492,5119356,571451,120434,24251,Puerto Rico,NaN
39784,Wyoming,2022,575705,487359,57696,28948,1702,Puerto Rico,NaN


In [72]:
long_format_migration_data.dtypes

,0
destination_state,object
year,object
population,int64
same_house,int64
same_state,int64
from_different_state_Total,int64
abroad_Total,int64
migration_type,object
number_of_people,float64


In [73]:
long_format_migration_data.to_csv('migrations_2010_to_2023.csv')